<a href="https://colab.research.google.com/github/klinhle2306/Education-Inequality/blob/main/Education_Inequality_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This project addresses inequality of educational opportunity in U.S. high schools. Here we will focus on average student performance on the ACT or SAT exams that students take as part of the college application process.

We expect a range of school performance on these exams, but is school performance predicted by socioeconomic factors?

## Import libraries

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split

# Model preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler


# Modeling
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Model metrics and analysis
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

## The data

This project utilizes two data sets. The primary data set is the EdGap data set from [EdGap.org](https://www.edgap.org/#5/37.875/-96.987). This data set from 2016 includes information about average ACT or SAT scores for schools and several socioeconomic characteristics of the school district. The secondary data set is basic information about each school from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp).





### EdGap data

All socioeconomic data (household income, unemployment, adult educational attainment, and family structure) are from the Census Bureau's American Community Survey. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) report that ACT and SAT score data is from each state's department of education or some other public data release.

### School information data

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about school.


## Load the data

Load the EdGap
 data set

In [60]:
edgap = pd.read_excel('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Load the school information data

In [61]:
!wget https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0

--2023-05-03 22:24:43--  https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv [following]
--2023-05-03 22:24:44--  https://www.dropbox.com/s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb2d796ad7eee3b53e671f7b014.dl.dropboxusercontent.com/cd/0/inline/B7ZNiB_gZh3avpNxTbmKtNlGJyWt_QPITnQ_zhs_1fvTnUvyLcuE-fLyaqhv4uiGmTgyRdiQ7i5wHvM4sm1fn1YDkEZ0wqslX__AVB0Va-gvJs7r56FxIFIzM7nVIuyqoqqcbGVIdPNSpYEGUQJ18yPWmHJtjnxEHwV0dAp3EzDgxw/file# [following]
--2023-05-03 22:24:44--  https://ucb2d796ad7eee3b53e671f7b014.dl.dropboxusercontent.com/cd/0/inline/B7ZNiB_

In [62]:
school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding= 'unicode_escape')

<ipython-input-62-54e0747c4ccf>:1: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding= 'unicode_escape')


## Explore the contents of the data sets

In [63]:
edgap.head()

,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [64]:
school_info.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported


In [65]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   NCESSCH School ID                                7986 non-null   int64  
 1   CT Unemployment Rate                             7972 non-null   float64
 2   CT Pct Adults with College Degree                7973 non-null   float64
 3   CT Pct Childre In Married Couple Family          7961 non-null   float64
 4   CT Median Household Income                       7966 non-null   float64
 5   School ACT average (or equivalent if SAT score)  7986 non-null   float64
 6   School Pct Free and Reduced Lunch                7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


In [66]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102183 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102183 non-null  object 
 1   FIPST                102183 non-null  int64  
 2   STATENAME            102183 non-null  object 
 3   ST                   102183 non-null  object 
 4   SCH_NAME             102183 non-null  object 
 5   LEA_NAME             102183 non-null  object 
 6   STATE_AGENCY_NO      102183 non-null  object 
 7   UNION                2533 non-null    float64
 8   ST_LEAID             102183 non-null  object 
 9   LEAID                102183 non-null  object 
 10  ST_SCHID             102183 non-null  object 
 11  NCESSCH              102181 non-null  float64
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

School info data set is much larger than the EdGap data set.

## Convert data types

I want to join the DataFrames using the identity of the school (NCESSCH) as the key.This has a different name and data types in the two DataFrames.

I cast the `NCESSCH` column in the `school_info` DataFrame as an `int64`. First, I need to drop rows where `NCESSCH` is a `NaN` value. 

In [67]:
school_info = school_info[school_info['NCESSCH'].isna() == False]

In [68]:
school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')

<ipython-input-68-006ea9997674>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')


In [69]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102181 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102181 non-null  object 
 1   FIPST                102181 non-null  int64  
 2   STATENAME            102181 non-null  object 
 3   ST                   102181 non-null  object 
 4   SCH_NAME             102181 non-null  object 
 5   LEA_NAME             102181 non-null  object 
 6   STATE_AGENCY_NO      102181 non-null  object 
 7   UNION                2531 non-null    float64
 8   ST_LEAID             102181 non-null  object 
 9   LEAID                102181 non-null  object 
 10  ST_SCHID             102181 non-null  object 
 11  NCESSCH              102181 non-null  int64  
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

## Select relevant subsets of the data

I only need the year, school identity, location, and school type information.

Keep the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL`

In [70]:
school_info = school_info[['SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']]

In [71]:
school_info.head()

,SCHOOL_YEAR,NCESSCH,MSTATE,MZIP,SCH_TYPE_TEXT,LEVEL
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


## Rename columns

In [72]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id", 
              "CT Pct Adults with College Degree":"percent_college",        
              "CT Unemployment Rate":"rate_unemployment", 
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})

In [73]:
school_info = school_info.rename(columns={'SCHOOL_YEAR':'year', 
                                          'NCESSCH':'id', 
                                          'MSTATE':'state',
                                          'MZIP':'zip_code',
                                          'SCH_TYPE_TEXT':'school_type',
                                          'LEVEL':'school_level'})

In [74]:
edgap.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [75]:
school_info.head()

,year,id,state,zip_code,school_type,school_level
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


## Join data frames 

In [76]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


In [77]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102181 entries, 0 to 102182
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   year          102181 non-null  object
 1   id            102181 non-null  int64 
 2   state         102181 non-null  object
 3   zip_code      102181 non-null  object
 4   school_type   102179 non-null  object
 5   school_level  102179 non-null  object
dtypes: int64(1), object(5)
memory usage: 5.5+ MB


In [78]:
df = edgap.merge(school_info, how = 'left', on='id')

In [79]:
df.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7986 entries, 0 to 7985
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
 7   year               7898 non-null   object 
 8   state              7898 non-null   object 
 9   zip_code           7898 non-null   object 
 10  school_type        7898 non-null   object 
 11  school_level       7898 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 811.1+ KB


In [81]:
df['year'].unique()

array(['2016-2017', nan], dtype=object)

In [82]:
df = df[['id',	'rate_unemployment',	'percent_college',	'percent_married',	'median_income',	'average_act',	'percent_lunch',	'state',	'zip_code',	'school_type',	'school_level']]

In [83]:
df

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,DE,19934,Regular School,High
...,...,...,...,...,...,...,...,...,...,...,...
7981,560530200294,0.034549,0.590780,0.720077,64891.0,18.500000,0.232156,WY,82902,Regular School,High
7982,560569000311,0.069538,0.633860,0.808841,44603.0,22.300000,0.083871,WY,82833,Regular School,High
7983,560569500360,0.035159,0.764566,0.776570,44896.0,21.500000,0.184829,WY,82801,Regular School,High
7984,560576200324,0.063877,0.670532,0.879324,92134.0,19.700000,0.112583,WY,82935,Regular School,High


In [84]:
df['school_level'].value_counts()

High            7230
Other            631
Not reported      35
Elementary         2
Name: school_level, dtype: int64

In [85]:
df['school_type'].value_counts()

Regular School                 7885
Alternative School               10
Special Education School          2
Career and Technical School       1
Name: school_type, dtype: int64

Schools may or may not require students to take ACT so that might impact the reported datas. 

I keep high schools, get rid of elementary schools

In [86]:
df = df.loc[df['school_level'] == 'High']
df['school_level'].value_counts()

High    7230
Name: school_level, dtype: int64

In [87]:
df.describe()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
count,7.230000e+03,7218.000000,7219.000000,7210.000000,7214.000000,7230.000000,7230.000000
mean,3.329050e+11,0.096437,0.572968,0.640475,52760.472692,20.288880,0.409447
std,1.333373e+11,0.056610,0.165067,0.191362,24365.505210,2.552661,0.235304
min,1.000016e+11,0.000000,0.091493,0.000000,4833.000000,-3.070818,-0.054545
25%,2.103645e+11,0.057915,0.454489,0.534494,37106.000000,18.800000,0.231863
50%,3.600087e+11,0.084249,0.559406,0.671713,47404.000000,20.500000,0.371203
75%,4.700148e+11,0.121022,0.680251,0.779260,62106.000000,22.000000,0.556800
max,5.605830e+11,0.590278,1.000000,1.000000,226181.000000,32.362637,0.998729


In [88]:
df['id'] = df['id'].map(str)

<ipython-input-88-323a7a4ca4a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['id'].map(str)


In [89]:
df.describe()

,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
count,7218.000000,7219.000000,7210.000000,7214.000000,7230.000000,7230.000000
mean,0.096437,0.572968,0.640475,52760.472692,20.288880,0.409447
std,0.056610,0.165067,0.191362,24365.505210,2.552661,0.235304
min,0.000000,0.091493,0.000000,4833.000000,-3.070818,-0.054545
25%,0.057915,0.454489,0.534494,37106.000000,18.800000,0.231863
50%,0.084249,0.559406,0.671713,47404.000000,20.500000,0.371203
75%,0.121022,0.680251,0.779260,62106.000000,22.000000,0.556800
max,0.590278,1.000000,1.000000,226181.000000,32.362637,0.998729


I get rid of ACT that is less than 1 and percent_lunch that are negative

In [90]:
df.loc[df['percent_lunch'] < 0.0, 'percent_lunch'] = np.nan
df = df[df['average_act'] >= 1.0]

In [91]:
df.describe()

,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
count,7215.000000,7216.000000,7207.000000,7211.000000,7227.000000,7207.000000
mean,0.096430,0.573075,0.640642,52771.280544,20.298577,0.410460
std,0.056616,0.165018,0.191221,24364.709754,2.508415,0.234442
min,0.000000,0.091493,0.000000,4833.000000,12.362637,0.000000
25%,0.057895,0.454604,0.534597,37115.000000,18.800000,0.232933
50%,0.084242,0.559444,0.671859,47410.000000,20.500000,0.371747
75%,0.120990,0.680315,0.779319,62109.000000,22.000000,0.557114
max,0.590278,1.000000,1.000000,226181.000000,32.362637,0.998729


In [92]:
df.isna().sum()

id                    0
rate_unemployment    12
percent_college      11
percent_married      20
median_income        16
average_act           0
percent_lunch        20
state                 0
zip_code              0
school_type           0
school_level          0
dtype: int64

In [93]:
df

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,DE,19934,Regular School,High
...,...,...,...,...,...,...,...,...,...,...,...
7981,560530200294,0.034549,0.590780,0.720077,64891.0,18.500000,0.232156,WY,82902,Regular School,High
7982,560569000311,0.069538,0.633860,0.808841,44603.0,22.300000,0.083871,WY,82833,Regular School,High
7983,560569500360,0.035159,0.764566,0.776570,44896.0,21.500000,0.184829,WY,82801,Regular School,High
7984,560576200324,0.063877,0.670532,0.879324,92134.0,19.700000,0.112583,WY,82935,Regular School,High


## Add a new attributes that may help with the analysis later

In [94]:
regions = {
    'SE': ['FL', 'GA', 'TN', 'NC'],
    'NE': ['MA', 'NY', 'NJ', 'PA', 'DE'],
    'LK': ['IL', 'IN', 'MI', 'KY', 'MO', 'WI', 'OH'],
    'S': ['LA', 'TX'],
    'N': ['WY'],
    'NW': ['WA']
}

In [95]:
_matching_lookup = {e: k for k, v in regions.items() for e in v}
df['region'] = df['state'].map(_matching_lookup)
df


,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,state,zip_code,school_type,school_level,region
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,DE,19804,Regular School,High,NE
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,DE,19709,Regular School,High,NE
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,DE,19709,Regular School,High,NE
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,DE,19958,Regular School,High,NE
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,DE,19934,Regular School,High,NE
...,...,...,...,...,...,...,...,...,...,...,...,...
7981,560530200294,0.034549,0.590780,0.720077,64891.0,18.500000,0.232156,WY,82902,Regular School,High,N
7982,560569000311,0.069538,0.633860,0.808841,44603.0,22.300000,0.083871,WY,82833,Regular School,High,N
7983,560569500360,0.035159,0.764566,0.776570,44896.0,21.500000,0.184829,WY,82801,Regular School,High,N
7984,560576200324,0.063877,0.670532,0.879324,92134.0,19.700000,0.112583,WY,82935,Regular School,High,N


#Split dataset into X and Y

In [96]:
X = df[['rate_unemployment',	'percent_college',	'percent_married',	'median_income',	'percent_lunch', 'region']]

In [97]:
X

,rate_unemployment,percent_college,percent_married,median_income,percent_lunch,region
0,0.117962,0.445283,0.346495,42820.0,0.066901,NE
1,0.063984,0.662765,0.767619,89320.0,0.112412,NE
2,0.056460,0.701864,0.713090,84140.0,0.096816,NE
3,0.044739,0.692062,0.641283,56500.0,0.296960,NE
4,0.077014,0.640060,0.834402,54015.0,0.262641,NE
...,...,...,...,...,...,...
7981,0.034549,0.590780,0.720077,64891.0,0.232156,N
7982,0.069538,0.633860,0.808841,44603.0,0.083871,N
7983,0.035159,0.764566,0.776570,44896.0,0.184829,N
7984,0.063877,0.670532,0.879324,92134.0,0.112583,N


In [98]:
y = df[['average_act']]

#Split data into training and testing sets

In [99]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [100]:
x_train

,rate_unemployment,percent_college,percent_married,median_income,percent_lunch,region
6050,0.096353,0.366770,0.659110,31009.0,0.644991,SE
7442,0.146809,0.624745,0.845479,64810.0,0.229216,NW
5593,0.053538,0.591367,0.854953,62621.0,0.123832,NE
3391,0.088304,0.544531,0.528000,40568.0,0.211094,LK
6300,0.039375,0.806250,0.957165,80429.0,0.197175,SE
...,...,...,...,...,...,...
4625,0.088907,0.559269,0.807556,42529.0,0.433164,SE
1974,0.072515,0.339232,0.473849,39464.0,0.425532,LK
4653,0.155347,0.577521,0.365327,35323.0,0.317700,SE
788,0.112046,0.569123,0.621442,50998.0,0.373802,SE


## Use MICE imputation on X train and test data

In [101]:
imputer = IterativeImputer()

In [102]:
imputer.fit(x_train.loc[:,'rate_unemployment':'percent_lunch'])

IterativeImputer()

In [103]:
x_train.loc[:, 'rate_unemployment':'percent_lunch'] = imputer.transform(x_train.loc[:,'rate_unemployment':'percent_lunch'])

In [104]:
x_train

,rate_unemployment,percent_college,percent_married,median_income,percent_lunch,region
6050,0.096353,0.366770,0.659110,31009.0,0.644991,SE
7442,0.146809,0.624745,0.845479,64810.0,0.229216,NW
5593,0.053538,0.591367,0.854953,62621.0,0.123832,NE
3391,0.088304,0.544531,0.528000,40568.0,0.211094,LK
6300,0.039375,0.806250,0.957165,80429.0,0.197175,SE
...,...,...,...,...,...,...
4625,0.088907,0.559269,0.807556,42529.0,0.433164,SE
1974,0.072515,0.339232,0.473849,39464.0,0.425532,LK
4653,0.155347,0.577521,0.365327,35323.0,0.317700,SE
788,0.112046,0.569123,0.621442,50998.0,0.373802,SE


In [105]:
x_train.isna().sum()

rate_unemployment    0
percent_college      0
percent_married      0
median_income        0
percent_lunch        0
region               0
dtype: int64

In [106]:
x_test.isna().sum()

rate_unemployment    0
percent_college      0
percent_married      2
median_income        1
percent_lunch        3
region               0
dtype: int64

In [107]:
x_test.loc[:,'rate_unemployment':'percent_lunch'] = imputer.transform(x_test.loc[:,'rate_unemployment':'percent_lunch'])

In [108]:
x_test.isna().sum()

rate_unemployment    0
percent_college      0
percent_married      0
median_income        0
percent_lunch        0
region               0
dtype: int64

In [109]:
y_train.isna().sum()

average_act    0
dtype: int64

In [110]:
y_test.isna().sum()

average_act    0
dtype: int64

In [111]:
train_data = x_train.join(y_train)
test_data = x_test.join(y_test)

In [112]:
train_data

,rate_unemployment,percent_college,percent_married,median_income,percent_lunch,region,average_act
6050,0.096353,0.366770,0.659110,31009.0,0.644991,SE,19.200000
7442,0.146809,0.624745,0.845479,64810.0,0.229216,NW,21.495726
5593,0.053538,0.591367,0.854953,62621.0,0.123832,NE,21.324786
3391,0.088304,0.544531,0.528000,40568.0,0.211094,LK,20.700000
6300,0.039375,0.806250,0.957165,80429.0,0.197175,SE,22.800000
...,...,...,...,...,...,...,...
4625,0.088907,0.559269,0.807556,42529.0,0.433164,SE,20.201465
1974,0.072515,0.339232,0.473849,39464.0,0.425532,LK,18.100000
4653,0.155347,0.577521,0.365327,35323.0,0.317700,SE,25.061050
788,0.112046,0.569123,0.621442,50998.0,0.373802,SE,19.224664


In [113]:
test_data

,rate_unemployment,percent_college,percent_married,median_income,percent_lunch,region,average_act
1849,0.095541,0.510452,0.737064,31283.0,0.432507,LK,18.500000
3736,0.212945,0.537480,0.233645,29725.0,0.714286,NE,17.124542
3503,0.131045,0.344122,0.648415,35128.0,0.365462,LK,21.600000
5444,0.084967,0.417903,0.758305,46458.0,0.301370,NE,20.372405
7156,0.117357,0.214347,0.675132,25208.0,0.895288,S,16.489621
...,...,...,...,...,...,...,...
7192,0.091786,0.768296,0.620643,72083.0,0.253731,S,23.278388
7635,0.028081,0.515789,0.722642,48517.0,0.203540,LK,23.500000
4402,0.116030,0.581487,0.451788,33832.0,0.289855,SE,21.813187
1056,0.210234,0.120885,0.687415,33449.0,0.963483,LK,16.600000


##Export clean CSV files

In [114]:
from google.colab import files

train_data.to_csv('clean_training_edGap.csv', encoding = 'utf-8-sig', index=False) 
files.download('clean_training_edGap.csv')

test_data.to_csv('clean_testing_edGap.csv', encoding = 'utf-8-sig', index=False) 
files.download('clean_testing_edGap.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>